In [4]:
# Import file reading and image packages
import os
import json
from PIL import Image
from PIL import ImageOps
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import sklearn
import random

%matplotlib inline

In [5]:
from tqdm import tqdm

# Import array and ML packages
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from keras.datasets import fashion_mnist
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Resize images, create XY pairs

def process_street_shop_data(category, set_type, width, height):
    dir = '/home/ubuntu/street2shop/images/' + category + '/' + set_type + '/'
    os.chdir(dir)
    user_list = []
    shop_list = []

    for image_name in tqdm(os.listdir()):
        if image_name[0] == 'i':
            resized_image = Image.open(image_name).resize((width, height))
            image_array = np.asarray(resized_image)
            user_list.append(image_array)
        if image_name[0] == 'o':
            resized_image = Image.open(image_name).resize((width, height))
            image_array = np.asarray(resized_image)
            shop_list.append(image_array)
    X = user_list
    Y = shop_list
    return (X, Y)

In [ ]:
# Perform resize and create XY pairs

width = 224
height = 224
train_X = [0]
train_Y = [0]
test_X = [0]
test_Y = [0]

# All available categories
#category_list = ['bags','belts','eyewear', 'footwear','hats','leggings','outerwear','pants','tops','dresses','skirts']

# Process subset for speed
category_list = ['tops']


for category in category_list:
    cur_train = process_street_shop_data(category, 'train', width, height)
    cur_test = process_street_shop_data(category, 'test', width, height)
    if len(train_X) == 1:
        train_X = cur_train[0]
        train_Y = cur_train[1]
        test_X = cur_test[0]
        test_Y = cur_test[1]
    else:
        train_X = np.concatenate((train_X, cur_train[0]))
        train_Y = np.concatenate((train_Y, cur_train[1]))
        test_X = np.concatenate((test_X, cur_test[0]))
        test_Y = np.concatenate((test_Y, cur_test[1]))
    print("Category complete\n")
    print("Train size: " + str(len(train_X)) + '\n')
    print("Test size: " + str(len(test_X)) + '\n')

In [ ]:
train_X = np.concatenate((train_X[:224], train_X[225:]))
train_Y = np.concatenate((train_Y[:224], train_Y[225:]))

In [ ]:
# Store the data in CSV format
os.chdir('/home/ubuntu/street2shop')
train_X = np.asarray(train_X)
test_X = np.asarray(test_X)
train_Y = np.asarray(train_Y)
test_Y = np.asarray(test_Y)
train_X_reshaped = train_X.reshape(train_X.shape[0], -1)
test_X_reshaped = test_X.reshape(test_X.shape[0], -1)
print(test_X_reshaped.shape)
train_Y_reshaped = train_Y.reshape(train_Y.shape[0], -1)
test_Y_reshaped = test_Y.reshape(test_Y.shape[0], -1)
np.savetxt("train_X_pants.csv",train_X_reshaped)
np.savetxt("test_X_pants.csv",test_X_reshaped)
np.savetxt("train_Y_pants.csv",train_Y_reshaped)
np.savetxt("test_Y_pants.csv", test_Y_reshaped)

In [6]:
# Load data from drive
os.chdir('/home/ubuntu/street2shop')
train_X_reshaped = np.loadtxt("train_X_pants.csv")
test_X_reshaped = np.loadtxt("test_X_pants.csv")
train_Y_reshaped = np.loadtxt("train_Y_pants.csv")
test_Y_reshaped = np.loadtxt("test_Y_pants.csv")

width = 224
height = 224
train_X = train_X_reshaped.reshape((train_X_reshaped.shape[0],width,height,3))
test_X = test_X_reshaped.reshape((test_X_reshaped.shape[0],width,height,3))
train_Y = train_Y_reshaped.reshape((train_Y_reshaped.shape[0],width,height,3))
test_Y = test_Y_reshaped.reshape((test_Y_reshaped.shape[0],width,height, 3))

In [ ]:
batch_size = len(train_X)
epochs = 300

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input
import numpy as np

baseModel = VGG16(weights=None, include_top=True, input_tensor=Input(shape=(128, 128, 3)), classes=len(train_X))
baseModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
print(baseModel.summary())
train_Y_model = np.array([x for x in range(len(train_X))])
train_Y_model = tf.keras.utils.to_categorical(train_Y_model)
print(train_Y_model.shape)
history = baseModel.fit(train_X, train_Y_model, batch_size=batch_size,epochs=epochs,verbose=1)

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    anchor, positive, negative = y_pred[:len(y_pred) // 3], y_pred[len(y_pred) // 3:2 * (len(y_pred) // 3)], y_pred[2 * (len(y_pred) // 3):]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))
    ### END CODE HERE ###
    
    return loss

In [ ]:
print(len(train_X))

In [ ]:
from keras.regularizers import l2

batch_size = (len(train_X) // 2) * 3
number_of_outputs = train_X.shape[0]
epochs = 10

street_to_shop_model = Sequential()
street_to_shop_model.add(Conv2D(64, kernel_size=(7, 7), strides=(2, 2), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(MaxPooling2D((3, 3),strides=(2, 2),padding='same'))
street_to_shop_model.add(BatchNormalization())

street_to_shop_model.add(Conv2D(64, kernel_size=(1, 1), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(BatchNormalization())
street_to_shop_model.add(MaxPooling2D((3, 3),strides=(2, 2),padding='same'))

street_to_shop_model.add(Conv2D(192, kernel_size=(1, 1),activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(Conv2D(192, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(MaxPooling2D((3, 3),strides=(2, 2),padding='same'))

street_to_shop_model.add(Conv2D(384, kernel_size=(1, 1), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(Conv2D(384, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(128,128,3)))

street_to_shop_model.add(Conv2D(256, kernel_size=(1, 1), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(128,128,3)))

street_to_shop_model.add(Conv2D(256, kernel_size=(1, 1), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(Conv2D(256, kernel_size=(3, 3), strides=(2, 2), activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(MaxPooling2D((3, 3),strides=(2, 2),padding='same'))

street_to_shop_model.add(Flatten())
street_to_shop_model.add(Dense(4096, activation='relu')) 
street_to_shop_model.add(Dropout(0.5))
street_to_shop_model.add(Dense(4096, activation='relu')) 
street_to_shop_model.add(Dropout(0.5))
street_to_shop_model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
street_to_shop_model.add(Dropout(0.5))
street_to_shop_model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))) 

street_to_shop_model.summary()
street_to_shop_model.compile(loss=triplet_loss, optimizer=keras.optimizers.Adam(learning_rate=0.01))

In [ ]:
batch_size = (len(train_X) // 2) * 3
number_of_outputs = train_X.shape[0]
epochs = 10

street_to_shop_model = Sequential()
street_to_shop_model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',padding='same',input_shape=(128,128,3)))
street_to_shop_model.add(MaxPooling2D((2, 2),padding='same'))
street_to_shop_model.add(Conv2D(32, (5, 5), activation='relu',padding='same'))
street_to_shop_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
street_to_shop_model.add(Conv2D(16, (4, 4), activation='relu',padding='same'))
street_to_shop_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# street_to_shop_model.add(Conv2D(8, (4, 4), activation='relu',padding='same'))
# street_to_shop_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
street_to_shop_model.add(Flatten())
street_to_shop_model.add(Dense(1024, activation='relu')) 
street_to_shop_model.add(Dropout(0.5))
street_to_shop_model.add(Dense(512, activation='relu'))     
street_to_shop_model.add(Dropout(0.5))
street_to_shop_model.add(Dense(256, activation='relu'))     
street_to_shop_model.add(Dropout(0.5))
street_to_shop_model.add(Dense(64, activation='relu'))
# street_to_shop_model.add(Dense(32, activation='relu'))
street_to_shop_model.add(Dense(12, activation='relu'))   
# street_to_shop_model.add(Dense(5, activation='relu'))     
# street_to_shop_model.add(Dropout(0.5))
# street_to_shop_model.add(Dense(number_of_outputs, activation='softmax'))

street_to_shop_model.summary()
street_to_shop_model.compile(loss=triplet_loss, optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
def generate_positive_and_negative_examples(X, Y):
    X_embed = street_to_shop_model([X])
    Y_embed = street_to_shop_model([Y])
    positive_examples = []
    negative_examples = []
    ex = []
    for i, embedding in enumerate(X_embed):
        positive_examples.append(Y[i])
        dists = tf.reduce_sum(
            tf.square(tf.subtract(
                tf.transpose(Y_embed), tf.tile(tf.reshape(embedding, (embedding.shape[0], 1)), tf.constant([1, Y_embed.shape[0]])))), axis=0
        )
        negative_examples.append(Y[(np.argsort(dists)[-1])])
#         rand_index = random.randint(0, X_embed.shape[0] - 1)
#         negative_examples.append(Y[rand_index])
#         ex.append((i, i))
#         ex.append((i, np.argsort(dists)[-1]))
#     print(ex)
    examples = np.concatenate((X, positive_examples, negative_examples))
    results = np.array([0 for _ in range(len(positive_examples) * 3)])
    return examples, results

In [ ]:
train_X_norm = train_X / 255
train_Y_norm = train_Y / 255

training_data, labels = generate_positive_and_negative_examples(train_X_norm, train_Y_norm)
# training_data, labels = sklearn.utils.shuffle(training_data, labels)
# print(training_data.shape, labels.shape)

In [ ]:
history = street_to_shop_model.fit(training_data, labels, batch_size=batch_size,epochs=epochs,verbose=1)

In [ ]:
for i in range(300):
    training_data, labels = generate_positive_and_negative_examples(train_X_norm, train_Y_norm)
    history = street_to_shop_model.fit(training_data, labels, batch_size=batch_size,epochs=epochs,verbose=1)

In [ ]:
street_to_shop_model.evaluate(train_X, train_Y_model)

In [10]:
# Performs cosine similarity checks
def street_to_shop(user_embedding, shop_embeddings, shop_images):
    """
    Implements cloth recognition by finding finding the corresponding shop product to the clothing on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings of the product database of shop pictures
    model -- our Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    product -- e-commerce image
    """
    
 ## Computes the target "encoding" for the image. 
    distances = []
    
    # Loop over the database.
    for i in range(len(shop_embeddings)):
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        db_enc = shop_embeddings[i]
        dist = np.linalg.norm(user_embedding - db_enc)
        distances.append(dist)
        

    ### END CODE HERE ###
    
    # if min_dist > 50:
    #     print("Not in the database.")
    # else:
    #     print ("The distance is " + str(min_dist))
        
    return np.array(distances)

In [ ]:
encode_embedding = Model(inputs= street_to_shop_model.input, outputs= street_to_shop_model.layers.output)

In [44]:
test_X_embedding = []
test_Y_embedding = []
for i in range(0, len(test_X), 15):
    test_X_embedding += list(embedding_model([test_X[i:i + 15]]))
for i in range(0, len(test_Y), 15):
    test_Y_embedding += list(embedding_model([test_Y[i:i + 15]]))
train_X_embedding = []
train_Y_embedding = []
for i in range(0, len(train_X), 15):
    train_X_embedding += list(embedding_model([train_X[i:i + 15]]))
for i in range(0, len(train_Y), 15):
    train_Y_embedding += list(embedding_model([train_Y[i:i + 15]]))

In [29]:
from tqdm import tqdm

predictions = []
correct = 0
total = 0
# test_X_embedding = embedding_model([test_X[:15]])
# test_Y_embedding = embedding_model([test_Y[:15]])

for i in tqdm(range(len(test_X_embedding))):
    image = test_X_embedding[i]
#     print(image.shape)
    dists = street_to_shop(image, test_Y_embedding, test_Y)
    print(np.argsort(dists))
    input()
    total += 1
    correct += (1 if i == np.argsort(dists)[0] else 0)

print('accuracy:', correct / total)

  0%|          | 0/627 [00:00<?, ?it/s]

[ 32 300 308 169 257  55 625 369  36 315 281  12 411 618 530 358  83 298
 396 268 130 473 420 514 120 609 171 301 508 419  18 340 110 554  38 287
 170 210 518 481 155 568 604 555 138 186  34  94 207  23  58 211 188 379
 333  57 489 524 222 220 311 515 204 350 243 383 303 184  93 545 439 143
 567 504 160 450 460 620 389  81   8 205 575 490 467 414  28 472 367 290
  87 227  92 323 444 223 166 304 569 622 355 580 457 357  49 386 330 197
 361 280 267 279 165  10 528 328 124   6 449 409 493 289 478  82 537 615
 366  60 320 119  13 101 276 176 491 500 399 194 291  66 193 131 402 480
 373 443 494 134 539 149 408 387 125 600 541 141 245 156  35   2 526 582
 181 509 574 503 459 332 416 364 499 187 225 592 248 272 591 343 109 576
 282 375 182 523 550 558 174 115 521 487 579 253 178 559   3  80 565 421
 133 543 217 585 201 432 285 116 368 198 540 306 342 258 247 605 544 112
  59 256 436 103 114 365  51 610 522 477 168  27  53  89 398 570  47 498
  22 381 458 424 177 278  24 534 435 505 337  45 62

  0%|          | 1/627 [00:06<1:09:28,  6.66s/it]

[ 55 411 530 281 369  36 315 625  12 303 618 308 396 298 268  83 358 130
 473 210 120 514 170 300  32  58 211 340 110 554 420 419 184 524 267 279
 280  18 483 350 204 257 169 518 155 481  57 442 602  87 138 186 555  34
  94 207 402 489 301 171 609 580 568 504 160 450  81 389  93 545 227 444
 223 287  38 508 333 131 357  49 620 467  23 550 490 600 515 460 472 367
 290 311  28 197 361 330 399 604 355 570  47 498 220 222 528  10 165 615
  92 487 304 166 569 622 585 457 373 541 141 205 575   8 386 439 115 323
 101  13 564 178 428 582 417  11 111 448 352 452  97  26 626 245 493 409
 414 119 174 499 188 379 328 591 109 343 332 416 181 503 574 509 459 383
 243 447 521 366 491 537 478  82 194 116 540 368 198 567 143 156  35   2
 526 176 282 321  22 449   6 124 552 495 284 480 261 217 276 436 523 387
 605 258  59 112 256 544 247  53 559  51 105 500 424 125 522 610 558 408
 133 421 565 180 374  50   5 566  52 583  80 432 285 543 337  45 477 193
  66 278 435  24 534 505 289 376 200 576 592 225 18

  0%|          | 2/627 [00:10<1:00:59,  5.86s/it]

[ 55 300  32 530 618  12 411 315  36 281 369 625  38 287 308  58  83 473
 130 358 298 268 396 169 257 340 554 110 414  18 420 490 210 419  57 518
 481 155 170 211 303 355 555  94 186 138  34 472 290 367  28 323 460 204
 514 120 467 383 243 350 508 609 301 171 125 568 620 207 399  23 223  93
 545 515 524 389  81 604 311 328 166 569 304 622 227 457 279 267 280 615
 592 225 187 272 248  87 222 220 489  92 184  49 357 580 409 493 574 332
 503 509 416 459 181 541 141 424 160 450 504 449 124   6 333 478  82 537
 480 143 567 500 444 550   2 526 156  35 499 386 197 361 330 402 582 559
 282 487 373 365 114 201 119 600 178 188 379 149 134 494 539 443 320  60
 447 285 432 115 439  13 101 523 528 165  10 174 112 605  59 544 258 256
 247 245 483 131 421 565 549 217 428  80 368 116 540 198 194 575 205   8
 359 395 438 364 491  66 193  22 334 176 289 276 263 404   9  33 378 543
 103 521 477 591 109 343   3 558 576 585 366 249 427 238 278 435 505 534
  24 624 129  27 436  53 168  97 352  11 417 452  2

  0%|          | 2/627 [00:29<2:34:21, 14.82s/it]


KeyboardInterrupt: Interrupted by user

In [ ]:
# Variational Autoencoder

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
latent_dim = 16

encoder_inputs = keras.Input(shape=(128, 128, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(32 * 32 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((32, 32, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 128 * 128
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [ ]:
train_X_norm = train_X / 255
train_Y_norm = train_Y / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(np.concatenate((train_X_norm, train_Y_norm)), epochs=1000, batch_size=256)

In [5]:
X_embed = embedding_model([train_X[:len(train_X) // 2]])
Y_embed = embedding_model([train_Y[:len(train_Y) // 2]])

def generate_embedding_similarity_dataset(X_embed, Y_embed):
    positive_examples = []
    negative_examples = []
    for i, embed in enumerate(X_embed):
        positive_examples.append(np.concatenate((embed, Y_embed[i])))
        generated = random.randint(0, len(X_embed) - 1)
        while generated == i:
            generated = random.randint(0, len(X_embed) - 1)
        negative_examples.append(np.concatenate((embed, Y_embed[generated])))
    examples = np.concatenate((positive_examples, negative_examples))
    labels = np.concatenate(([1 for _ in range(len(X_embed))], [0 for _ in range(len(X_embed))]))
    examples, labels = sklearn.utils.shuffle(examples, labels)
    return examples, labels

examples, label = generate_embedding_similarity_dataset(X_embed, Y_embed)

NameError: name 'embedding_model' is not defined

In [ ]:
examples[0].shape

In [45]:
# Learn a similarity metric
batch_size = (len(train_X) // 2) * 3
epochs = 100

similarity_model = Sequential()
similarity_model.add(Dense(256, activation='relu', input_dim=8192)) 
similarity_model.add(Dense(256, activation='relu'))     
similarity_model.add(Dense(1, activation='sigmoid'))

similarity_model.summary()
similarity_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 2,163,457
Trainable params: 2,163,457
Non-trainable params: 0
_________________________________________________________________


In [47]:
for i in range(10):
    examples, label = generate_embedding_similarity_dataset(train_X_embedding, train_Y_embedding)
    similarity_model.fit(examples, label, batch_size=batch_size,epochs=epochs)

Epoch 1/100
2/2 [==============================] - 0s 5ms/step - loss: 3.5944 - accuracy: 0.5019
Epoch 2/100
2/2 [==============================] - 0s 4ms/step - loss: 2.3057 - accuracy: 0.5115
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.9131 - accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 0.8286 - accuracy: 0.4974
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.8160 - accuracy: 0.4987
Epoch 6/100
2/2 [==============================] - 0s 3ms/step - loss: 0.7436 - accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 0.7061 - accuracy: 0.5166
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 0.7699 - accuracy: 0.4994
Epoch 9/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6991 - accuracy: 0.5480
Epoch 10/100
2/2 [==============================] - 0s 3ms/step - loss: 0.7412 - accuracy: 0.5077
Epoch 11/100
2/2 [===========

2/2 [==============================] - 0s 3ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 85/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0198 - accuracy: 1.0000
Epoch 86/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0187 - accuracy: 1.0000
Epoch 87/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0175 - accuracy: 1.0000
Epoch 88/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 89/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 90/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 91/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0139 - accuracy: 1.0000
Epoch 92/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0132 - accuracy: 1.0000
Epoch 93/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0125 - accuracy: 1.0000
Epoch 94/100
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 0.0557 - accuracy: 0.9898
Epoch 68/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0529 - accuracy: 0.9898
Epoch 69/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0502 - accuracy: 0.9923
Epoch 70/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0475 - accuracy: 0.9930
Epoch 71/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0445 - accuracy: 0.9936
Epoch 72/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0424 - accuracy: 0.9955
Epoch 73/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0399 - accuracy: 0.9949
Epoch 74/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0381 - accuracy: 0.9955
Epoch 75/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0363 - accuracy: 0.9968
Epoch 76/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0342 - accuracy: 0.9968
Epoch 77/100
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 0.0226 - accuracy: 1.0000
Epoch 51/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 52/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 53/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 54/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0191 - accuracy: 1.0000
Epoch 55/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0184 - accuracy: 1.0000
Epoch 56/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0177 - accuracy: 1.0000
Epoch 57/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0170 - accuracy: 1.0000
Epoch 58/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0164 - accuracy: 1.0000
Epoch 59/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0158 - accuracy: 1.0000
Epoch 60/100
2/2 [===============

2/2 [==============================] - 0s 4ms/step - loss: 0.0132 - accuracy: 1.0000
Epoch 34/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 35/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 36/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 37/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 38/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 39/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0104 - accuracy: 1.0000
Epoch 40/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0100 - accuracy: 1.0000
Epoch 41/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 42/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 43/100
2/2 [===============

...

2/2 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 66/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 67/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 68/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 69/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 70/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 71/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 72/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 73/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 74/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 75/100
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 49/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 50/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 51/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 52/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 53/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 54/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 55/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 56/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 57/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 58/100
2/2 [===============

2/2 [==============================] - 0s 4ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 32/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 33/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 34/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 35/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 36/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 37/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 38/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 39/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 40/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 41/100
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 0.0236 - accuracy: 0.9981
Epoch 15/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0184 - accuracy: 0.9987
Epoch 16/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0143 - accuracy: 0.9987
Epoch 17/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0130 - accuracy: 0.9987
Epoch 18/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0125 - accuracy: 0.9994
Epoch 19/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0114 - accuracy: 0.9987
Epoch 20/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0098 - accuracy: 0.9994
Epoch 21/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0090 - accuracy: 0.9994
Epoch 22/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0082 - accuracy: 0.9994
Epoch 23/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0075 - accuracy: 0.9994
Epoch 24/100
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 0.9994
Epoch 98/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 0.9994
Epoch 99/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 0.9994
Epoch 100/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 0.9994


In [48]:
# X_embed_test = embedding_model([test_X])
# Y_embed_test = embedding_model([test_Y])

def generate_embedding_similarity_dataset(X_embed, Y_embed):
    positive_examples = []
    negative_examples = []
    for i, embed in enumerate(X_embed):
        positive_examples.append(np.concatenate((embed, Y_embed[i])))
        generated = random.randint(0, len(X_embed) - 1)
        while generated == i:
            generated = random.randint(0, len(X_embed) - 1)
        negative_examples.append(np.concatenate((embed, Y_embed[generated])))
    examples = np.concatenate((positive_examples, negative_examples))
    labels = np.concatenate(([1 for _ in range(len(X_embed))], [0 for _ in range(len(X_embed))]))
    examples, labels = sklearn.utils.shuffle(examples, labels)
    return examples, labels

examples, label = generate_embedding_similarity_dataset(test_X_embedding, test_Y_embedding)

In [49]:
similarity_model.evaluate(examples, label)

40/40 [==============================] - 0s 2ms/step - loss: 4.6186 - accuracy: 0.5104


[4.618595600128174, 0.5103667974472046]

In [ ]:
from tqdm import tqdm

predictions = []
correct = 0
total = 0
# test_X_embedding = street_to_shop_model([test_X])
# test_Y_embedding = street_to_shop_model([test_Y])
for i in tqdm(range(len(test_X_embedding))):
    probs = []
    concat_mat = np.concatenate((tf.tile(test_X_embedding[i], tf.constant([len(test_Y_embedding)], tf.int32)), test_Y_embedding)).reshape(1, 8192)
    prob = similarity_model.predict(concat_mat)[0][0]
    probs.append(prob)
    probs_indexes_sorted = np.argsort(probs)
    correct += (1 if probs_indexes_sorted[-1] == i else 0)
    total += 1
    
print('accuracy:', correct / total)

  0%|          | 0/627 [00:00<?, ?it/s]

In [7]:
from keras.applications.vgg16 import VGG16
model = VGG16()

In [8]:
embedding_model = Model(inputs=model.input, outputs= model.layers[-2].output)

In [9]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     